In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

# Arguments

In [2]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [3]:
pretrained = './log/pggan/128/b0/checkpoint.pth.tar'
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style1/l2sp_pggan' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_pggan')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes)
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.010000
1/13 | Total:0:00:01 | ETA:0:00:17 | Loss:1.350160837173462 | top1:78.125
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.7821485942060297 | top1:73.01136779785156
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.7714977165063223 | top1:72.39583587646484


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


26/26 | Total:0:00:17 | ETA:0:00:00 | Loss:0.9672942299109238 | top1:48.73076629638672
107/107 | Total:0:00:26 | ETA:0:00:00 | Loss:0.01919553858863416 | top1:99.7352066040039

Epoch: [2 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.3619804382324219 | top1:84.375
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.5151450173421339 | top1:72.7272720336914
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.5131735801696777 | top1:72.91667175292969

Epoch: [3 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.4767477512359619 | top1:65.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.47527501528913324 | top1:72.44318389892578
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.4750666543841362 | top1:72.91667175292969

Epoch: [4 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.26738104224205017 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.44220882654190063 | top1:75.0
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.4404876381158829 | top1:75.52083587

1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.26223504543304443 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.2768478447740728 | top1:87.21591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.27824048201243085 | top1:87.23958587646484

Epoch: [32 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.3963708281517029 | top1:78.125
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.30935280837795953 | top1:83.80682373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.29959936439991 | top1:84.63542175292969

Epoch: [33 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.22051459550857544 | top1:90.625
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.26636896892027423 | top1:88.06818389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.27321676661570865 | top1:88.02083587646484

Epoch: [34 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.23366203904151917 | top1:84.375
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.31239874254573474 | top1:84.9

1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.14873753488063812 | top1:93.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.17385761710730466 | top1:92.32955169677734
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.18455157801508904 | top1:91.66667175292969

Epoch: [61 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.16421249508857727 | top1:93.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.202900383960117 | top1:91.19318389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.1987154223024845 | top1:91.14583587646484

Epoch: [62 | 5000] LR: 0.024000
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.1337186098098755 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2777668677947738 | top1:88.63636779785156
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2781613574673732 | top1:89.0625

Epoch: [63 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.4043896198272705 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.23976918919519943 | top1:90.05682373046875
1

...

11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.004035250199112025 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.003787076255927483 | top1:99.73958587646484

Epoch: [4859 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.05559321492910385 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.005886555395343087 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.005424432456493378 | top1:99.73958587646484

Epoch: [4860 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.0009370073676109314 | top1:100.0
11/13 | Total:0:00:01 | ETA:0:00:01 | Loss:0.006768996742638675 | top1:99.43182373046875
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.010311673705776533 | top1:99.21875

Epoch: [4861 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0003967359662055969 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.025796028023416347 | top1:99.43182373046875
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.025197

1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0010367408394813538 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.009558074514974247 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.008808393962681293 | top1:99.73958587646484

Epoch: [4889 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0004351809620857239 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0007680745964700526 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0012748249185582001 | top1:100.0

Epoch: [4890 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.06692472100257874 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.009703872217373415 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.00894459430128336 | top1:99.73958587646484

Epoch: [4891 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0003149956464767456 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.012066460807215084 | top1:99.715

11/13 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0021017023616216397 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.001997779356315732 | top1:100.0

Epoch: [4918 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0010466724634170532 | top1:100.0
11/13 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0026141295040195637 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0025213706927994886 | top1:99.73958587646484

Epoch: [4919 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0003855377435684204 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002447219988839193 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002263847350453337 | top1:100.0

Epoch: [4920 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.0002719983458518982 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.01506359926001592 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.014367776612440744 | top1:99.4791717529296

1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.0006984174251556396 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.001261205327781764 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0012384822281698387 | top1:100.0

Epoch: [4948 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.00017218291759490967 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.001997666687450626 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0018863610457628965 | top1:100.0

Epoch: [4949 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0010731667280197144 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0009071088650009849 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0008543947090705236 | top1:100.0

Epoch: [4950 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0003506019711494446 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.013255921615795656 | top1:99.71591186523438
13/13 | Total:0:00:03 | ETA:0:0

1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.00039518624544143677 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.004389675334095955 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.004053835291415453 | top1:99.73958587646484

Epoch: [4977 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.0026040785014629364 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0012458457865498283 | top1:100.0
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0012574195861816406 | top1:100.0

Epoch: [4978 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.00028933584690093994 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002961866388266737 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.00279063219204545 | top1:99.73958587646484

Epoch: [4979 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.0006994828581809998 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0064079419455745 | top1:99.71